Trades
+ Адрес хранилища данных: points
Получение данных
Адрес хранилища данных: данные
+ download data
+ read file
Получение словаря датафремов трейдов
+ Данные: словарь датафреймов
+ method(type): type
Преобразования к таймфрему оси чарта
+ Датафрейм: Агрегированный датафрейм по таймфрему
+ Агрегация экзекьюшинов с разницей времени < 5s
+ Агрегация по таймфреймам
Рисование экзекьюшина на чарте
+ Dataframe: chart
+ plotly
главная программа
"Перенесение трейдов на график"
главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)
Рисование чарта
Рисование трейдов
Candlestick
Хранилище данных: chart object
подготовка данных для графика
+ адрес хранилища данных: строка
Получение данных
+ адресс файла (строка): содержимое файла
+ чтение данных
Подготовка датафрейма
Объект данных: строка
+ csv to Dataframe
+ Convert Data
+ Prepare columns
Выбор времени
Датафрейм : рабочий датафрейм
+ обрезаем датафрейм до нужных данных
Получение таймфрейма
+ Dataframe : Dataframe
+ трансформация датафрейма 
   в нужный таймфрейм
Plotting
+ Dataframe: Plott
ploly candlestick
Чарт


деф главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)

Неправильно работает группировка, так как в Qty неверно отображается объем

In [1]:
import pandas as pd
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
import win32com.client

plotly.tools.set_credentials_file(username='mkf', api_key='crwJntc1jJsAdaHKiDPL')

In [2]:
def fix_excel(trades_file):
    import win32com.client

    xl=win32com.client.Dispatch("Excel.Application")
    xl.Application.DisplayAlerts = False # disables Excel pop up message (for saving the file)
    wb = xl.Workbooks.Open(Filename=file_excel)
    wb.SaveAs(file_excel)
    wb.Close
    xl.Application.DisplayAlerts = True

In [3]:
def open_excel(path):
    try:
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    except CompDocError:
        fix_excel(path)
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    return open_file

In [4]:
def normalise_formats(storage_of_executions):
    important_columns = ['Order Id', 'Time', 'B/S', 'Qty', 'Price', 'Gross', 'Net', 'Route']
    copy_storage_of_executions = storage_of_executions.copy()
    for date, tickers in storage_of_executions.items():
        for ticker, executions in tickers.items():
            new_df = executions[important_columns]
            new_df.insert(0, 'Date_Time', pd.to_datetime((date + ' ' + new_df['Time']), dayfirst=False))
            new_df.insert(1, 'Ticker', ticker)
            new_df['BP_used'] = new_df['Price'] * new_df['Qty']
            new_df['Qty'] = new_df.apply(lambda x: x['Qty'] if new_df['B/S'][x.name] in ['B', 'C'] else -x['Qty'], axis=1)
            #new_df.insert(6, 'Pos_size', new_df['Qty'].cumsum())
            new_df.reset_index(inplace=True)
            copy_storage_of_executions[date][ticker] = group_by_order_final(new_df)
    return copy_storage_of_executions

In [100]:
def group_by_order(executions):
    round_sum = lambda x: round(sum(x), 2)
    executions_by_order = executions.groupby('Order Id').agg(({'index': 'first', 'Ticker' : 'first','Time' : 'first', 
                                                               'Date_Time' : 'first', 'B/S' : 'first', 'Qty' : 'sum',
                                                               #'Pos_size' : 'sum', 
                                                               'Gross' : round_sum, 'Net' : round_sum, 'Route' : 'first',
                                                               'BP_used': 'sum'}))
    executions_by_order['Price'] = abs(round(executions_by_order['BP_used'] / executions_by_order['Qty'], 3))
    return executions_by_order

In [6]:
def group_by_order_final(execution):
    new_group_by_orger = group_by_order(pd.DataFrame(columns=execution.columns))
    one_group = pd.DataFrame(columns=execution.columns)
    for i in range(len(execution)):
        if one_group.empty:
            one_group = one_group.append(execution.iloc[i, :])
        if i == len(execution) - 1:
            new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
            one_group = pd.DataFrame(columns=execution.columns)
        else:
            try:
                if execution['Order Id'][i] == execution['Order Id'][i+1]:
                    one_group = one_group.append(execution.iloc[i, :])
                else:
                    new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                    one_group = pd.DataFrame(columns=execution.columns)
            except:
                new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                one_group = pd.DataFrame(columns=execution.columns)
    new_group_by_orger.insert(6, 'Pos_size', new_group_by_orger['Qty'].cumsum())
    return new_group_by_orger

In [7]:
def preprocessing_trades(storage_of_executions):
    file_parsing = {}
    excel_df = storage_of_executions
    reindex = {a : b for a, b in zip(range(22), excel_df.iloc[2, :])}
    excel_df = excel_df.rename(columns=reindex)
    for i in range(len(excel_df)):
        if pd.notna(excel_df.iloc[i][0]):
            if re.match(r'\d{1,2}/\d{1,2}/\d{4}', excel_df.iloc[i][0]):
                date = excel_df.iloc[i][0]
                file_parsing[date] = {}
                continue

            elif re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0]):
                ticker = re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0])[1]
                file_parsing[date][ticker] = pd.DataFrame()
                continue

            elif excel_df.iloc[i][0] == 'Time':

                file_parsing[date][ticker].reindex(columns=excel_df.iloc[i])
                columns = excel_df.iloc[i]
                continue

            elif re.match(r'\d\d:\d\d:\d\d', excel_df.iloc[i][0]):
                file_parsing[date][ticker] = file_parsing[date][ticker].append(excel_df.iloc[i, :])
    date_ticker_exec = normalise_formats(file_parsing)
    return date_ticker_exec

In [ ]:
def create_points(data):
    days_trades = preprocessing_trades(file_name)
 

In [4]:
def make_chart_df(chart_file):
    pass

In [114]:
def create_chart_df(chart_file):
    chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
    chart_df[['Open', 'High', 'Low', 'Close', 'PC', 'Vol']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC', 'Vol']].apply(lambda x:  round(x, 2))
    chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))
    return chart_df

In [6]:
def draw_trades(trades_df):
    pass

In [7]:
def create_picture(chart_df, trades_df):
    create_chart(chart_df)
    draw_trades(trades_df)

In [9]:
trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

trades_file = r'C:\Users\Kir\Documents\07060212-2018-08-06-to-2018-08-09-detailed.xlsx'
chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

In [15]:
if __name__ == '__main__':
    trades_df = processing_trades(trades_file)
    chart_df = make_chart_df(chart_file)
    create_picture(chart_df, trades_df)

In [64]:
chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
chart_df[['Open', 'High', 'Low', 'Close', 'PC']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC']].apply(lambda x:  round(x, 2))
chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))

In [66]:
chart_df.head()

,Date_Time,Date,Time,Open,High,Low,Close,PC
0,2016-07-15 15:55:00,15.07.2016,15:55:00,215.80,215.96,215.79,215.81,216.12
1,2016-07-15 15:50:00,15.07.2016,15:50:00,215.84,215.90,215.79,215.80,216.12
2,2016-07-15 15:45:00,15.07.2016,15:45:00,215.89,215.94,215.83,215.83,216.12
3,2016-07-15 15:40:00,15.07.2016,15:40:00,216.04,216.04,215.88,215.90,216.12
4,2016-07-15 15:35:00,15.07.2016,15:35:00,216.06,216.09,215.98,216.03,216.12


In [75]:
trace = go.Candlestick(x=chart_df.Date_Time,
                       open=chart_df.Open,
                       high=chart_df.High,
                       low=chart_df.Low,
                       close=chart_df.Close)
data = [trace]
#py.iplot(data)

In [91]:
excel_df = pd.read_excel(r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls', encoding='utf-16', header=None)

In [49]:
def transform_one_df_to_date_axis(date, ticker, detailed_df):
    #important_columns = ['Time', 'B/S', 'Qty', 'Price', 'Gross', 'Net', 'Order Id', 'Route']
    executions = detailed_df[date][ticker][important_columns]
    
    


def transform_to_date_axis(file_parsing, tickers=None):
    if not tickers:
        
        days_trades = preprocessing_trades(file_name)
    


In [101]:
%%time
date = '8/6/2018'
ticker = 'ARLO'

#trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
#chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

opened_file = open_excel(trades_file)
date_tickers_exec = preprocessing_trades(opened_file)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Wall time: 22.1 s


In [103]:
date_tickers_exec[date][ticker]['Net'].sum()

-913.23

In [104]:
date_tickers_exec[date][ticker]

,index,Ticker,Time,Date_Time,B/S,Qty,Pos_size,Gross,Net,Route,BP_used,Price
Order Id,,,,,,,,,,,,
121819,3,ARLO,09:22:20,2018-08-06 09:22:20,B,176.0,176.0,0.00,-1.25,CROX,3967.04,22.540
123167,5,ARLO,09:24:03,2018-08-06 09:24:03,B,100.0,276.0,0.00,-0.76,ARCA,2231.00,22.310
128583,6,ARLO,09:31:18,2018-08-06 09:31:18,B,100.0,376.0,0.00,-0.56,NYSE,2155.00,21.550
129324,7,ARLO,09:31:41,2018-08-06 09:31:41,S,-200.0,176.0,-233.76,-235.36,EDGX,4272.00,21.360
128682,8,ARLO,09:31:47,2018-08-06 09:31:47,S,-200.0,-24.0,-286.00,-287.62,EDGX,4176.00,20.880
137504,10,ARLO,09:54:53,2018-08-06 09:54:53,S,-200.0,-224.0,0.00,-1.60,ARCA,4158.00,20.790
138844,11,ARLO,10:01:10,2018-08-06 10:01:10,S,-100.0,-324.0,0.00,-0.81,ARCA,2087.50,20.875
138963,12,ARLO,10:27:30,2018-08-06 10:27:30,B,500.0,176.0,-381.50,-385.27,EDGX,10785.00,21.570


In [102]:
date_tickers_exec['8/6/2018']['BRK/B']['Net'].sum()

143.28999999999996

In [258]:
execution = date_tickers_exec['8/6/2018']['ARLO']#['Time'].apply(lambda x: dt.datetime.strptime(x, '%H:%M:%S')

In [200]:
#group_by_order(pd.DataFrame(columns=execution.columns)).append(executions_by_order)

In [287]:
execution
#execution['Qty'] = [execution['Qty'] if execution['B/S'] in ['B','C'] else -execution['Qty']]
execution['Qty'] = execution.apply(lambda x: x['Qty'] if execution['B/S'][x.name] in ['B', 'C'] else -x['Qty'], axis=1)
execution['Pos_side'] = execution['Qty'].cumsum()

In [288]:
execution

,index,Date_Time,Ticker,Order Id,Time,B/S,Qty,Price,Gross,Net,Route,BP_used,Pos_side
0,3,2018-08-06 09:22:20,ARLO,121819,09:22:20,B,88.0,22.540,0.00,-0.626,CROX,1983.52,88.0
1,4,2018-08-06 09:22:20,ARLO,121819,09:22:20,B,112.0,22.520,0.00,-0.850,ARCA,2522.24,200.0
2,5,2018-08-06 09:24:03,ARLO,123167,09:24:03,B,100.0,22.310,0.00,-0.760,ARCA,2231.00,300.0
3,6,2018-08-06 09:31:18,ARLO,128583,09:31:18,B,100.0,21.550,0.00,-0.560,NYSE,2155.00,400.0
4,7,2018-08-06 09:31:41,ARLO,129324,09:31:41,S,-200.0,21.360,-233.76,-235.360,EDGX,4272.00,200.0
5,8,2018-08-06 09:31:47,ARLO,128682,09:31:47,S,-100.0,20.880,-143.00,-143.810,EDGX,2088.00,100.0
6,9,2018-08-06 09:31:47,ARLO,128682,09:31:47,S,-100.0,20.890,-66.00,-66.810,EDGX,2089.00,0.0
7,10,2018-08-06 09:54:53,ARLO,137504,09:54:53,S,-200.0,20.790,0.00,-1.600,ARCA,4158.00,-200.0
8,11,2018-08-06 10:01:10,ARLO,138844,10:01:10,S,-100.0,20.875,0.00,-0.810,ARCA,2087.50,-300.0
9,12,2018-08-06 10:27:30,ARLO,138963,10:27:30,B,250.0,21.570,-190.75,-192.635,EDGX,5392.50,-50.0


In [41]:
new_group_by_orger

NameError: name 'new_group_by_orger' is not defined

,Date_Time,Date,Time,Bar#,Bar Index,Tick Range,Open,High,Low,Close,PL,PC,MA,PH,Vol
0,2018-11-09 15:28:00,09.11.2018,15:28:00,20683/20683,0,0,12.90,12.90,12.90,12.90,13.4588,13.79,12.9,14.02,200
1,2018-11-09 15:27:00,09.11.2018,15:27:00,20682/20683,-1,1,12.91,12.91,12.90,12.90,13.4588,13.79,12.9,14.02,400
2,2018-11-09 15:24:00,09.11.2018,15:24:00,20681/20683,-2,3,12.90,12.92,12.89,12.90,13.4588,13.79,12.9,14.02,2400
3,2018-11-09 15:23:00,09.11.2018,15:23:00,20680/20683,-3,0,12.89,12.89,12.89,12.89,13.4588,13.79,12.9,14.02,300
4,2018-11-09 15:20:00,09.11.2018,15:20:00,20679/20683,-4,5,12.92,12.92,12.87,12.87,13.4588,13.79,12.9,14.02,4300


In [63]:
date_of_trade.date()

datetime.date(2018, 8, 6)

In [105]:
drowing_trades = date_tickers_exec['8/6/2018']['ARLO']

In [35]:
drowing_trades.Date_Time.apply(lambda x: x.replace(second=0))

Order Id
121819   2018-08-06 09:22:00
123167   2018-08-06 09:24:00
128583   2018-08-06 09:31:00
129324   2018-08-06 09:31:00
128682   2018-08-06 09:31:00
137504   2018-08-06 09:54:00
138844   2018-08-06 10:01:00
138963   2018-08-06 10:27:00
Name: Date_Time, dtype: datetime64[ns]

In [52]:
b = a.replace(day=a.day+1)

In [46]:
a = date_tickers_exec[date]['ARLO'].Date_Time[0].date()

In [92]:
chart_df = create_chart_df(r'C:\Users\Kir\Documents\ARLO_06.08.18.csv')

In [93]:
date_of_trade = date_tickers_exec[date]['ARLO'].Date_Time[0]
needed_chart = chart_df[(chart_df.Date_Time.apply(lambda x: x.date()) >= date_of_trade.date()) &
      (chart_df.Date_Time.apply(lambda x: x.date()) < date_of_trade.date().replace(day=date_of_trade.day+1))]

In [ ]:
func = lambda x: x.date()
chart_df

In [94]:
needed_chart

,Date_Time,Date,Time,Bar#,Bar Index,Tick Range,Open,High,Low,Close,PL,PC,MA,PH,Vol
20099,2018-08-06 19:45:00,06.08.2018,19:45:00,584/20683,-20099,0,20.92,20.92,20.92,20.92,18.3,22.1,20.92,22.45,100
20100,2018-08-06 19:41:00,06.08.2018,19:41:00,583/20683,-20100,0,21.10,21.10,21.10,21.10,18.3,22.1,20.92,22.45,200
20101,2018-08-06 18:24:00,06.08.2018,18:24:00,582/20683,-20101,0,20.75,20.75,20.75,20.75,18.3,22.1,20.90,22.45,250
20102,2018-08-06 18:22:00,06.08.2018,18:22:00,581/20683,-20102,0,20.79,20.79,20.79,20.79,18.3,22.1,20.92,22.45,241
20103,2018-08-06 18:13:00,06.08.2018,18:13:00,580/20683,-20103,0,20.79,20.79,20.79,20.79,18.3,22.1,20.93,22.45,125
20104,2018-08-06 17:57:00,06.08.2018,17:57:00,579/20683,-20104,0,21.00,21.00,21.00,21.00,18.3,22.1,20.94,22.45,203
20105,2018-08-06 17:53:00,06.08.2018,17:53:00,578/20683,-20105,0,21.00,21.00,21.00,21.00,18.3,22.1,20.94,22.45,267
20106,2018-08-06 17:16:00,06.08.2018,17:16:00,577/20683,-20106,0,21.00,21.00,21.00,21.00,18.3,22.1,20.93,22.45,100
20107,2018-08-06 16:49:00,06.08.2018,16:49:00,576/20683,-20107,0,20.89,20.89,20.89,20.89,18.3,22.1,20.92,22.45,200
20108,2018-08-06 16:19:00,06.08.2018,16:19:00,575/20683,-20108,0,21.29,21.29,21.29,21.29,18.3,22.1,20.93,22.45,100


In [106]:
drowing_trades

,index,Ticker,Time,Date_Time,B/S,Qty,Pos_size,Gross,Net,Route,BP_used,Price
Order Id,,,,,,,,,,,,
121819,3,ARLO,09:22:20,2018-08-06 09:22:20,B,176.0,176.0,0.00,-1.25,CROX,3967.04,22.540
123167,5,ARLO,09:24:03,2018-08-06 09:24:03,B,100.0,276.0,0.00,-0.76,ARCA,2231.00,22.310
128583,6,ARLO,09:31:18,2018-08-06 09:31:18,B,100.0,376.0,0.00,-0.56,NYSE,2155.00,21.550
129324,7,ARLO,09:31:41,2018-08-06 09:31:41,S,-200.0,176.0,-233.76,-235.36,EDGX,4272.00,21.360
128682,8,ARLO,09:31:47,2018-08-06 09:31:47,S,-200.0,-24.0,-286.00,-287.62,EDGX,4176.00,20.880
137504,10,ARLO,09:54:53,2018-08-06 09:54:53,S,-200.0,-224.0,0.00,-1.60,ARCA,4158.00,20.790
138844,11,ARLO,10:01:10,2018-08-06 10:01:10,S,-100.0,-324.0,0.00,-0.81,ARCA,2087.50,20.875
138963,12,ARLO,10:27:30,2018-08-06 10:27:30,B,500.0,176.0,-381.50,-385.27,EDGX,10785.00,21.570


In [110]:
drowing_trades['Candle_Time'] = drowing_trades.Date_Time.apply(lambda x: candle_time(x, 1))

In [79]:
def candle_time(series, timeframe):
    #print(series, type(series))
    minute = series.minute - series.minute % timeframe
    return dt.datetime(series.year, series.month, series.day, series.hour, minute)
    
    #for i in len(dataframe):
    #    dataframe.iloc[i, 3]
    #dataframe.insert(0, 'Candle_Time', )

In [121]:
drowing_trades['B/S'].isin(['B', 'C'])

Order Id
121819     True
123167     True
128583     True
129324    False
128682    False
137504    False
138844    False
138963     True
Name: B/S, dtype: bool

In [126]:
buy_executions = drowing_trades.loc[drowing_trades['B/S'].isin(['B', 'C'])]

In [124]:
sell_executions = drowing_trades.loc[drowing_trades['B/S'].isin(['S', 'T'])]

In [129]:
buy_executions

,index,Ticker,Time,Date_Time,B/S,Qty,Pos_size,Gross,Net,Route,BP_used,Price,Candle_Time
Order Id,,,,,,,,,,,,,
121819,3,ARLO,09:22:20,2018-08-06 09:22:20,B,176.0,176.0,0.0,-1.25,CROX,3967.04,22.54,2018-08-06 09:22:00
123167,5,ARLO,09:24:03,2018-08-06 09:24:03,B,100.0,276.0,0.0,-0.76,ARCA,2231.00,22.31,2018-08-06 09:24:00
128583,6,ARLO,09:31:18,2018-08-06 09:31:18,B,100.0,376.0,0.0,-0.56,NYSE,2155.00,21.55,2018-08-06 09:31:00
138963,12,ARLO,10:27:30,2018-08-06 10:27:30,B,500.0,176.0,-381.5,-385.27,EDGX,10785.00,21.57,2018-08-06 10:27:00


In [128]:
chart = go.Candlestick(x=needed_chart.Date_Time,
                       open=needed_chart.Open,
                       high=needed_chart.High,
                       low=needed_chart.Low,
                       close=needed_chart.Close)

buy_trace = go.Scatter(x=buy_executions.Candle_Time, y=buy_executions.Price, mode='markers')
sell_trace = go.Scatter(x=sell_executions.Candle_Time, y=sell_executions.Price, mode='markers')
data = [trace, buy_trace, sell_trace]
py.iplot(data)

In [ ]:
py.iplot(data)